In [1]:
import numpy as np
import pandas as pd
import scipy.optimize as optimize
from fffit.utils import (
    shuffle_and_split,
    values_scaled_to_real,
)
import sys
sys.path.append("../")


from utils.r143a import R143aConstants
R143a = R143aConstants()

In [2]:
# top_liq = pd.read_csv("../csv/r143a-density-iter1-liquid-params.csv", delimiter = ",", index_col = 0)
# top_vap = pd.read_csv("../csv/r143a-density-iter1-vapor-params.csv", delimiter = ",", index_col = 0)

In [3]:
iternum = 2
csv_path = "/scratch365/mcarlozo/HFC_143a_FFO_FF/r143a/analysis/csv/"
in_csv_name = "r143a-density-iter" + str(iternum) + "-results.csv"
out_csv_name = "r143a-density-iter" + str(iternum + 1) + "-params.csv"
out_top_liquid_csv_name = "r143a-density-iter" + str(iternum ) + "-liquid-params.csv"
out_top_vapor_csv_name = "r143a-density-iter" + str(iternum ) + "-vapor-params.csv"

top_liq = pd.read_csv(csv_path + out_top_liquid_csv_name, delimiter = ",", index_col = 0)
top_vap = pd.read_csv(csv_path + out_top_vapor_csv_name, delimiter = ",", index_col = 0)

In [4]:
top_liq = top_liq.reset_index(drop=True)
top_vap = top_vap.reset_index(drop=True)
# print(len(top_vap))
# print(len(top_liq))

dist_guess = 1
dist_guess_l = 1
dist_guess_v = 0.8

dist_seed = 115
bounds = [(0.000001,None)]
target_num = 100

In [5]:
# print(top_liq)

In [6]:
def opt_dist(distance, top_samples, constants, target_num, rand_seed = None, eval = False):
    """
    Calculates the distance between points such that exactly a target number of points are chosen for the next iteration
    
    Parameters:
    -----------
        distance: float, The allowable minimum distance between points
        top_samples: pandas data frame, Collection of top liquid/vapor sampes
        constants: utils.r143a.R143aConstants, contains the infromation for a certain refrigerant
        target_num: int, the number of samples to choose next
        rand_seed: int, the seed number to use: None by default
        eval: bool, Determines whether error is calculated or new_points is returned
    
    Returns:
        error: float, The squared error between the target value and number of new_points
        OR
        new_points: pandas data frame, a pandas data frame containing the number of points to be used 
    """
    if len(top_samples) <= target_num:
        print("Trying dist =", distance)
        
    top_samp0 = top_samples
    if rand_seed != None:
        np.random.seed(rand_seed)
    new_points = pd.DataFrame()
    discarded_points = pd.DataFrame(columns=top_samples.columns)
    while len(top_samples > 0):
        # Shuffle the pareto points
        top_samples = top_samples.sample(frac=1)
        new_points = new_points.append(top_samples.iloc[[0]])
        # Remove anything within distance
        l1_norm = np.sum(
            np.abs(
                top_samples[list(constants.param_names)].values
                - new_points[list(constants.param_names)].iloc[[-1]].values
            ),
            axis=1,
        )
        points_to_remove = np.where(l1_norm < distance)[0]
        discarded_points = discarded_points.append(
            top_samples.iloc[points_to_remove]
        )
        top_samples.drop(
            index=top_samples.index[points_to_remove], inplace=True
        )
    error = target_num - len(new_points)
    
#     print("Error = ",error)
#     return error
    if eval == True:
        return new_points
    else:
        return error

In [7]:
args_l = (top_liq ,R143a, target_num, dist_seed)
solution_l = optimize.root(opt_dist, dist_guess_l, args = args_l, method = "broyden1", options={'maxiter': 50})
dist_opt_l = solution_l.x
new_points_l = opt_dist(dist_opt_l, top_liq, R143a, target_num, rand_seed=dist_seed , eval = True)

while int(len(new_points_l)) != int(target_num):
    dist_guess_l = dist_opt_l
    solution_l = optimize.root(opt_dist, dist_guess_l, args = args_l, method = "broyden1", options={'maxiter': 50})
    dist_opt_l = solution_l.x
    new_points_l = opt_dist(dist_opt_l, top_liq, R143a, target_num, rand_seed=dist_seed , eval = True)
    dist_seed += 1
    print("Trying seed", dist_seed)
    
print("Optimal Liquid Distance:",dist_opt_l)

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/hfcs-fffit/lib/python3.7/site-packages/scipy/optimize/nonlin.py:1016: RuntimeWarning: divide by zero encountered in true_divide
  d = v / vdot(df, v)


Optimal Liquid Distance: 1.2863865224930684


In [8]:
args_v = (top_vap ,R143a, target_num, dist_seed)
solution_v = optimize.root(opt_dist, dist_guess_v, args = args_v, method = "broyden1", options={"maxiter": 50})
dist_opt_v = solution_v.x
new_points_v = opt_dist(dist_opt_v, top_vap, R143a, target_num, rand_seed=dist_seed , eval = True)

while int(len(new_points_v)) != int(target_num):
    dist_guess_v = dist_opt_v
    solution_v = optimize.root(opt_dist, dist_guess_v, args = args_v, method = "broyden1", options={'maxiter': 50})
    dist_opt_v = solution_v.x
    new_points_v = opt_dist(dist_opt_v, top_vap, R143a, target_num, rand_seed=dist_seed , eval = True) 
    dist_seed += 1
    print("Trying seed", dist_seed)
    
print("Optimal Vapor Distance:",dist_opt_v)

Trying dist = 0.8
Trying dist = 1.3
Trying dist = 0.8806279530559367
Trying dist = 0.8172457699001667
Trying dist = 0.811016020973718
Trying dist = 0.8094396727734201
Trying dist = 0.8042596728318604
Trying dist = 0.8021298364159303
Trying dist = 1.3
Trying dist = 0.14285714285714302
Trying dist = -0.0017857142857141128


KeyboardInterrupt: 